# General Assembly Data Science Immersive -- Elliot Cohen
## Part 1
### Python Coding and Data Set

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

import sklearn
sklearn.__version__

'0.18.2'

In [2]:
# Load in the data file and header file provided
header_url = 'https://gist.githubusercontent.com/jeff-boykin/b5c536467c30d66ab97cd1f5c9a3497d/raw/5233c792af49c9b78f20c35d5cd729e1307a7df7/field_names.txt'
header_list = pd.read_csv(header_url, header=None, squeeze=True).tolist();

data_url = 'https://gist.githubusercontent.com/jeff-boykin/b5c536467c30d66ab97cd1f5c9a3497d/raw/5233c792af49c9b78f20c35d5cd729e1307a7df7/breast-cancer.csv'
data = pd.read_csv(data_url, header=None, names=header_list, index_col='ID')

In [3]:
# Comment on any steps you might take to evaluate or transform the dataset.
data.apply(pd.Categorical).describe()

,diagnosis,radius_mean,radius_sd_error,radius_worst,texture_mean,texture_sd_error,texture_worst,perimeter_mean,perimeter_sd_error,perimeter_worst,...,concavity_worst,concave_points_mean,concave_points_sd_error,concave_points_worst,symmetry_mean,symmetry_sd_error,symmetry_worst,fractal_dimension_mean,fractal_dimension_sd_error,fractal_dimension_worst
count,569,569.00,569.00,569.0,569.0,569.0000,569.0000,569.0,569.0,569.0000,...,569.00,569.00,569.0,569.0,569.0000,569.0000,569.0,569.0,569.0000,569.00000
unique,2,456.00,479.00,522.0,539.0,474.0000,537.0000,537.0,542.0,432.0000,...,457.00,511.00,514.0,544.0,411.0000,529.0000,539.0,492.0,500.0000,535.00000
top,B,12.34,18.22,134.7,512.2,0.1007,0.1147,0.0,0.0,0.1714,...,12.36,27.26,101.7,826.4,0.1216,0.1486,0.0,0.0,0.2383,0.07427
freq,357,4.00,3.00,3.0,3.0,5.0000,3.0000,13.0,13.0,4.0000,...,5.00,3.00,3.0,2.0,4.0000,3.0000,13.0,13.0,3.0000,3.00000


In [4]:
# Compute the mean and median smoothness and compactness for benign and malignant tumors
data.filter(regex='smoothness|compactness|diagnosis').groupby('diagnosis').agg(['mean', 'median'])

smoothness_mean         smoothness_sd_error          \
                     mean  median                mean  median   
diagnosis                                                       
B                2.000321  1.8510           21.135148  19.630   
M                4.323929  3.6795           72.672406  58.455   

          smoothness_worst           compactness_mean           \
                      mean    median             mean   median   
diagnosis                                                        
B                 0.007196  0.006530         0.021438  0.01631   
M                 0.006780  0.006209         0.032281  0.02859   

          compactness_sd_error           compactness_worst            
                          mean    median              mean    median  
diagnosis                                                             
B                     0.025997  0.018400          0.009858  0.009061  
M                     0.041824  0.037125          0.015060  0.014205

In [5]:
# Do the groups differ? Explain how you would identify this.
grouped = data.groupby('diagnosis')
malignant = data.loc[grouped.groups['M']]
benign = data.loc[grouped.groups['B']]

t, p = ttest_ind(malignant['smoothness_mean'], benign['smoothness_mean']) # T-test for the means of two independent samples.
print('{}: t statitic = {:.6f} p value = {:.9f}'.format('smoothness', t, p))

t, p = ttest_ind(malignant['compactness_mean'], benign['compactness_mean']) # T-test for the means of two independent samples.
print('{}: t statitic = {:.6f} p value = {:.9f}'.format('compactness', t, p))

smoothness: t statitic = 15.934158 p value = 0.000000000
compactness: t statitic = 7.297077 p value = 0.000000000


Grouping the cancer data by diagnosis, and summarizing by smoothness and compactness features, we see a discernable difference in measures of central tendancy (e.g. mean and median). To determine if this difference is statistically significant, we conduct a two-sided T-test with a null hypothesis that the two independent samples are identically distributed.

Results indicate that we can reject the null hypothesis that mean cell nuclei smoothness for benign and malignant tupors are the same, based on a p_value approaching zero. Similarly for compactness, we can reject the null hypthoses that benign and malignant cell nuclei compactness are the same.

In [6]:
# Write a function to generate bootstrap samples of the data.
def create_bootstrap_samples_from_dataframe(dataframe, n_samples=1000):
    assert isinstance(dataframe, pd.core.frame.DataFrame), 'input data must be a pandas dataframe'
    return dataframe.sample(n=n_samples, replace=True)

In [7]:
# example usage
bootstrap_data = create_bootstrap_samples_from_dataframe(data, n_samples=10000)

print('original smoothness for benign cells: {:.6f}'.format(data[data['diagnosis']=='B']['smoothness_mean'].mean()))
print('resampled smoothness for benign cells: {:.6f}'.format(bootstrap_data[bootstrap_data['diagnosis']=='B']['smoothness_mean'].mean()))

original smoothness for benign cells: 2.000321
resampled smoothness for benign cells: 2.006036


### Exploratory Analysis
Identify 2-3 variables that are predictive of a malignant tumor.  
Display the relationship visually and write 1-2 sentences explaining the relationship.

In [8]:
# divide data into predictors (X) and predictand (y)
x_train = data.ix[:, data.columns != 'diagnosis']
y_train = data['diagnosis']

/Users/ecohen/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


In [9]:
# preprocess data to give each feature a mean of 0 and a std dev of 1. 
# This will help us later to determine which features are more important than others.
from sklearn import preprocessing


scaler = preprocessing.StandardScaler().fit(x_train)
print('column means:')
print(scaler.mean_)
print('column stds:')
print(scaler.scale_)
x_train_scaled = scaler.transform(x_train)
print('these should be zero:')
print(x_train_scaled.mean(axis=0))
print('these should be one:')
print(x_train_scaled.std(axis=0))

column means:
[  1.41272917e+01   1.92896485e+01   9.19690334e+01   6.54889104e+02
   9.63602812e-02   1.04340984e-01   8.87993158e-02   4.89191459e-02
   1.81161863e-01   6.27976098e-02   4.05172056e-01   1.21685343e+00
   2.86605923e+00   4.03370791e+01   7.04097891e-03   2.54781388e-02
   3.18937163e-02   1.17961371e-02   2.05422988e-02   3.79490387e-03
   1.62691898e+01   2.56772232e+01   1.07261213e+02   8.80583128e+02
   1.32368594e-01   2.54265044e-01   2.72188483e-01   1.14606223e-01
   2.90075571e-01   8.39458172e-02]
column stds:
[  3.52095076e+00   4.29725464e+00   2.42776193e+01   3.51604754e+02
   1.40517641e-02   5.27663291e-02   7.96497253e-02   3.87687325e-02
   2.73901809e-02   7.05415588e-03   2.77068942e-01   5.51163427e-01
   2.02007710e+00   4.54510134e+01   2.99987837e-03   1.78924359e-02
   3.01595231e-02   6.16486075e-03   8.25910439e-03   2.64374475e-03
   4.82899258e+00   6.14085432e+00   3.35730016e+01   5.68856459e+02
   2.28123569e-02   1.57198171e-01   2.0

In [10]:
# Next use ridge regression, choosing the hyperparameter based on 10-fold cross-validation.
from sklearn.model_selection import KFold, GridSearchCV
from sklearn import linear_model


classifier = linear_model.LogisticRegression(penalty='l1', class_weight='balanced')
kfold = KFold(n_splits=10, shuffle=False)

param_grid = {'C': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.25, 0.3, 1.0, 3.0, 5.0, 
                    7.0, 10.0, 20.0, 500.0, 550., 600.0, 700., 800., 1000.0, 1100.0, 1200., 1500.]}

cv = GridSearchCV(classifier,
                  param_grid=param_grid,
                  scoring='neg_log_loss',
                  cv=kfold,
                  verbose=1)

cv.fit(x_train_scaled, y_train.values)
print("BEST: {} {}".format(cv.best_params_, cv.best_score_))

Fitting 10 folds for each of 23 candidates, totalling 230 fits
BEST: {'C': 0.3} -0.096947666635038


[Parallel(n_jobs=1)]: Done 230 out of 230 | elapsed:  1.4min finished


In [11]:
final_classifier = cv.best_estimator_
final_classifier.fit(x_train_scaled, y_train.values)
coefficients = final_classifier.coef_
intercept = final_classifier.intercept_
final_classifier.predict_proba(x_train_scaled)

array([[  5.30015881e-07,   9.99999470e-01],
       [  5.45969495e-04,   9.99454031e-01],
       [  1.58713990e-05,   9.99984129e-01],
       ..., 
       [  2.50526301e-02,   9.74947370e-01],
       [  3.46357522e-08,   9.99999965e-01],
       [  9.99601826e-01,   3.98173782e-04]])

In [ ]:
data[data['diagnosis']=='M'].plot(kind='density', subplots=True, layout=(10,3), sharex=False, figsize=(18,18))
data[data['diagnosis']=='B'].plot(kind='density', subplots=True, layout=(10,3), sharex=False, figsize=(18,18))

In [ ]:
from sklearn.cluster import KMeans
X = np.array([[1, 2], [1, 4], [1, 0], [4, 2], [4, 4], [4, 0]])
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
print(kmeans.labels_)
print(kmeans.predict([[0, 0], [4, 4]]))
print(kmeans.cluster_centers_)

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

In [ ]:
import numpy as np
from scipy.stats import ttest_ind, ttest_ind_from_stats
from scipy.special import stdtr

np.random.seed(1)

# Create sample data.
a = np.random.randn(40)
b = 4*np.random.randn(50)

# Use scipy.stats.ttest_ind.
t, p = ttest_ind(a, b, equal_var=False)
print("ttest_ind:            t = %g  p = %g" % (t, p))

# Compute the descriptive statistics of a and b.
abar = a.mean()
avar = a.var(ddof=1)
na = a.size
adof = na - 1

bbar = b.mean()
bvar = b.var(ddof=1)
nb = b.size
bdof = nb - 1

# Use scipy.stats.ttest_ind_from_stats.
t2, p2 = ttest_ind_from_stats(abar, np.sqrt(avar), na,
                              bbar, np.sqrt(bvar), nb,
                              equal_var=False)
print("ttest_ind_from_stats: t = %g  p = %g" % (t2, p2))

# Use the formulas directly.
tf = (abar - bbar) / np.sqrt(avar/na + bvar/nb)
dof = (avar/na + bvar/nb)**2 / (avar**2/(na**2*adof) + bvar**2/(nb**2*bdof))
pf = 2*stdtr(dof, -np.abs(tf))

print("formula:              t = %g  p = %g" % (tf, pf))